In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [2]:
df1 = pd.read_csv('Train_UWu5bXk.txt',sep=',')
df2 = pd.read_csv('Test_u94Q5KV.txt',sep=',')

In [3]:
item_id = df2.Item_Identifier
outlet_id = df2.Outlet_Identifier

In [4]:
df1['data']  = 0
df2['data']  = 1
df2['Item_Outlet_Sales'] = np.nan

In [5]:
df = df1.append(df2)

In [6]:
df['Item_Fat_Content'].loc[(df['Item_Fat_Content']=='low fat') | (df['Item_Fat_Content']=='LF')] = 'Low Fat'
df['Item_Fat_Content'].loc[(df['Item_Fat_Content']=='reg')] = "Regular"
df['Item_Fat_Content'] = df['Item_Fat_Content'].map({'Regular':0,'Low Fat':1}).astype(int)

df['Item_Identifier'] = df['Item_Identifier'].apply(lambda x: x[0:2])
df['Item_Identifier'] = df['Item_Identifier'].map({'FD' : 0,'NC' : 1,'DR' : 2}).astype(int)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Item_Type'] = le.fit_transform(df['Item_Type']).astype(int)

df['Item_Visibility'] = df['Item_Visibility'].replace(0,np.mean(df.Item_Visibility))

df['Outlet_Identifier'] = df['Outlet_Identifier'].apply(lambda x: x[4:]).astype(int)

df['Outlet_Location_Type'] = df['Outlet_Location_Type'].apply(lambda x: x[5]).astype(int)

df['Outlet_Size'] = df['Outlet_Size'].map({'Small':0,'Medium':1,'High':2})
df['Outlet_Type'] = df['Outlet_Type'].map({'Supermarket Type1':0,'Supermarket Type2':1,'Supermarket Type3':2,'Grocery Store':3}).astype(int)

In [7]:
# num_var = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['int64','float64']]
# cat_var = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['object']]

In [8]:
from missingpy import MissForest,KNNImputer
mf = MissForest()
data = mf.fit_transform(df)
#knn = KNNImputer()
#data = knn.fit_transform(df)
df_miss = pd.DataFrame(data,columns=df.columns)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [9]:
df_miss.isnull().sum()

Item_Fat_Content             0
Item_Identifier              0
Item_MRP                     0
Item_Outlet_Sales            0
Item_Type                    0
Item_Visibility              0
Item_Weight                  0
Outlet_Establishment_Year    0
Outlet_Identifier            0
Outlet_Location_Type         0
Outlet_Size                  0
Outlet_Type                  0
data                         0
dtype: int64

In [10]:
df_miss.Outlet_Size = df_miss.Outlet_Size.round()

In [11]:
# With one hot encoding
df_copy = df_miss.copy()

Item_Fat_Content_dummies = pd.get_dummies(df_miss.Item_Fat_Content,drop_first=True)
Item_Identifier_dummies = pd.get_dummies(df_miss.Item_Identifier,drop_first=True)
Item_Type_dummies = pd.get_dummies(df_miss.Item_Type,drop_first=True)
Outlet_Establishment_Year_dummies =pd.get_dummies(df_miss.Outlet_Establishment_Year,drop_first=True)
Outlet_Identifier_dummies = pd.get_dummies(df_miss.Outlet_Identifier,drop_first=True)
Outlet_Location_Type_dummies = pd.get_dummies(df_miss.Outlet_Location_Type,drop_first=True)
Outlet_Size_dummies = pd.get_dummies(df_miss.Outlet_Size,drop_first=True)
Outlet_Type_dummies = pd.get_dummies(df_miss.Outlet_Type,drop_first=True)
df_copy.drop(['Item_Fat_Content','Item_Identifier','Item_Type',
                'Outlet_Establishment_Year','Outlet_Identifier','Outlet_Location_Type',
               'Outlet_Size','Outlet_Type'],axis = 1, inplace = True)
df_copy = pd.concat([df_copy,Item_Fat_Content_dummies,Item_Identifier_dummies,
                    Item_Type_dummies,Outlet_Establishment_Year_dummies,Outlet_Location_Type_dummies,
                    Outlet_Size_dummies,Outlet_Type_dummies],axis = 1)

In [62]:
df_train = df_miss[df_miss['data'] == 0]

df_train = df_train.drop('data',axis = 1)

df_test = df_miss[df_miss['data']== 1]

df_test = df_test.drop('data',axis = 1)

df_test = df_test.drop('Item_Outlet_Sales',axis = 1)

In [311]:
df_train = df_copy[df_copy['data'] == 0]

df_train = df_train.drop('data',axis = 1)

df_test = df_copy[df_copy['data']== 1]

df_test = df_test.drop('data',axis = 1)

df_test = df_test.drop('Item_Outlet_Sales',axis = 1)

# Modeling

In [63]:
X = df_train.drop('Item_Outlet_Sales',axis =1)

In [64]:
y = df_train.Item_Outlet_Sales

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.3,random_state=1)

In [67]:
from sklearn.metrics import mean_squared_error

# Gradient Boosting

In [68]:
from sklearn.ensemble import GradientBoostingRegressor

GB = GradientBoostingRegressor(n_estimators=600,learning_rate=0.01)

GB.fit(X_train,y_train)

np.sqrt(mean_squared_error(y_val,GB.predict(X_val)))

1066.3086266007454

In [153]:
from sklearn.model_selection import GridSearchCV
params = {'n_estimators': [100,200,300,400], 'learning_rate': [0.1, 0.5, 1.],
           'subsample': [0.05,1.0],
           'max_features': [0.05,1.0] }

grid_search = GridSearchCV(GB,param_grid=params,cv=5)

grid_search.fit(X_train,y_train)

best_gb = grid_search.best_estimator_

best_gb.fit(X_train,y_train)
ypred = best_gb.predict(X_train)
best_gb_ypred = best_gb.predict(X_val)

print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(y_train,ypred)))
#print('R2 value: ',r2_score(y_train,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(y_val,best_gb_ypred)))
#print('R2 value: ',r2_score(y_val,best_gb_ypred)*100)

TRAIN
RMSE:  1025.525080357911
TEST
RMSE:  1069.784950677551


In [188]:
# y_pred = GB.predict(df_test)
# res = pd.DataFrame({'Item_Identifier':item_id,'Outlet_Identifier':outlet_id,'Item_Outlet_Sales':y_pred})
# res.to_csv('Gradient.csv',index=False)

# Bagging Regressor

In [69]:
from sklearn.ensemble import BaggingRegressor

BR = BaggingRegressor(base_estimator=et)

BR.fit(X_train,y_train)

np.sqrt(mean_squared_error(y_val,BR.predict(X_val)))

1067.2106728740687

In [190]:
# y_pred = BR.predict(df_test)
# res = pd.DataFrame({'Item_Identifier':item_id,'Outlet_Identifier':outlet_id,'Item_Outlet_Sales':y_pred})
# res.to_csv('Bagging.csv',index=False)

In [55]:
from tpot import TPOTRegressor

In [56]:
tpt = TPOTRegressor()
tpt.fit(X_train,y_train)

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=100,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=100,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=0, warm_start=False)

In [57]:
np.sqrt(mean_squared_error(y_val,tpt.predict(X_val)))

1070.6603923672722

# Extra Trees

In [73]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor(bootstrap=True, max_features=0.8, min_samples_leaf=19, min_samples_split=16, n_estimators=500)
et.fit(X_train,y_train)
np.sqrt(mean_squared_error(y_val,et.predict(X_val)))

1064.6285847730023

In [74]:
# y_pred = et.predict(df_test)
# res = pd.DataFrame({'Item_Identifier':item_id,'Outlet_Identifier':outlet_id,'Item_Outlet_Sales':y_pred})
# res.to_csv('et_3.csv',index=False)